# Лабораторная работа №5

### Распознавание объектов на фотографиях

##### Задачи

1. Ознакомиться со сверточными нейронными сетями
2. Изучить построение модели в Keras в функциональном виде
3. Изучить работу слоя разреживания (Dropout)

### Цель работы

Распознавание объектов на фотографиях (Object Recognition in Photographs) CIFAR-10 (классификация небольших изображений по десяти классам: самолет, автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль и грузовик).

In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

In [2]:
batch_size = 32 
num_epochs = 200 
kernel_size = 3 # использование ядра 3x3 
pool_size = 2 #   использование объединения 2х2 
conv_depth_1 = 32 #  32 ядра на слой преобразования
conv_depth_2 = 64 # переход на 64 после первого уровня объединения
drop_prob_1 = 0.25 #  вероятность 0,25
drop_prob_2 = 0.5 #  в плотном слое вероятность 0,5
hidden_size = 512 # в плотном слое 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 
num_train, depth, height, width = X_train.shape 
num_test = X_test.shape[0] 
num_classes = np.unique(y_train).shape[0]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_train) 

Y_train = np_utils.to_categorical(y_train, num_classes) 
Y_test = np_utils.to_categorical(y_test, num_classes) 


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)  #Для предотвращения переобучения
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding - сохранять
#размер исходного изображения, рисунок дополняется нулями по краям. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
 
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=0, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

313/313 [==============================] - 1s 2ms/step - loss: 558.5317 - accuracy: 0.2041


[558.5316772460938, 0.20409999787807465]

Без слоя DropOut

In [3]:
batch_size = 32 
num_epochs = 200 
kernel_size = 3 # использование ядра 3x3 
pool_size = 2 #   использование объединения 2х2 
conv_depth_1 = 32 #  32 ядра на слой преобразования
conv_depth_2 = 64 # переход на 64 после первого уровня объединения
drop_prob_1 = 0.25 #  вероятность 0,25
drop_prob_2 = 0.5 #  в плотном слое вероятность 0,5
hidden_size = 512 # в плотном слое 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 
num_train, depth, height, width = X_train.shape 
num_test = X_test.shape[0] 
num_classes = np.unique(y_train).shape[0]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_train) 

Y_train = np_utils.to_categorical(y_train, num_classes) 
Y_test = np_utils.to_categorical(y_test, num_classes) 


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
#drop_1 = Dropout(drop_prob_1)(pool_1)  #Для предотвращения переобучения
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(pool_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding - сохранять
#размер исходного изображения, рисунок дополняется нулями по краям. 
#drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(pool_2)
hidden = Dense(hidden_size, activation='relu')(flat)

#drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(hidden)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=0, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

313/313 [==============================] - 1s 2ms/step - loss: 3493.9905 - accuracy: 0.2157


[3493.990478515625, 0.21570000052452087]

In [4]:
batch_size = 32 
num_epochs = 200 
kernel_size = 3 
pool_size = 2 
conv_depth_1 = 16 
conv_depth_2 = 32 
drop_prob_1 = 0.25 
drop_prob_2 = 0.5 
hidden_size = 512 

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 
num_train, depth, height, width = X_train.shape 
num_test = X_test.shape[0] 
num_classes = np.unique(y_train).shape[0]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_train) 

Y_train = np_utils.to_categorical(y_train, num_classes) 
Y_test = np_utils.to_categorical(y_test, num_classes) 


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)  #Для предотвращения переобучения
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding - сохранять
#размер исходного изображения, рисунок дополняется нулями по краям. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=0, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

313/313 [==============================] - 1s 2ms/step - loss: 269.1863 - accuracy: 0.2174


[269.1863098144531, 0.21739999949932098]

In [5]:
batch_size = 32 
num_epochs = 200 
kernel_size = 3 # использование ядра 3x3 
pool_size = 2 #   использование объединения 2х2 
conv_depth_1 = 128 #  32 ядра на слой преобразования
conv_depth_2 = 256 # переход на 64 после первого уровня объединения
drop_prob_1 = 0.25 #  вероятность 0,25
drop_prob_2 = 0.5 #  в плотном слое вероятность 0,5
hidden_size = 512 # в плотном слое 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 
num_train, depth, height, width = X_train.shape 
num_test = X_test.shape[0] 
num_classes = np.unique(y_train).shape[0]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_train) 

Y_train = np_utils.to_categorical(y_train, num_classes) 
Y_test = np_utils.to_categorical(y_test, num_classes) 


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)  #Для предотвращения переобучения
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding - сохранять
#размер исходного изображения, рисунок дополняется нулями по краям. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=0, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

313/313 [==============================] - 2s 6ms/step - loss: 625.1826 - accuracy: 0.2587


[625.1825561523438, 0.25870001316070557]

In [6]:
batch_size = 32 
num_epochs = 200 
kernel_size = 1 # использование ядра 3x3 
pool_size = 2 #   использование объединения 2х2 
conv_depth_1 = 32 #  32 ядра на слой преобразования
conv_depth_2 = 64 # переход на 64 после первого уровня объединения
drop_prob_1 = 0.25 #  вероятность 0,25
drop_prob_2 = 0.5 #  в плотном слое вероятность 0,5
hidden_size = 512 # в плотном слое 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 
num_train, depth, height, width = X_train.shape 
num_test = X_test.shape[0] 
num_classes = np.unique(y_train).shape[0]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_train) 

Y_train = np_utils.to_categorical(y_train, num_classes) 
Y_test = np_utils.to_categorical(y_test, num_classes) 


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)  #Для предотвращения переобучения
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding - сохранять
#размер исходного изображения, рисунок дополняется нулями по краям. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=0, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

313/313 [==============================] - 3s 11ms/step - loss: 7567.1392 - accuracy: 0.1015


[7567.13916015625, 0.1014999970793724]

In [7]:
batch_size = 32 
num_epochs = 200 
kernel_size = 5 # использование ядра 3x3 
pool_size = 2 #   использование объединения 2х2 
conv_depth_1 = 32 #  32 ядра на слой преобразования
conv_depth_2 = 64 # переход на 64 после первого уровня объединения
drop_prob_1 = 0.25 #  вероятность 0,25
drop_prob_2 = 0.5 #  в плотном слое вероятность 0,5
hidden_size = 512 # в плотном слое 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 
num_train, depth, height, width = X_train.shape 
num_test = X_test.shape[0] 
num_classes = np.unique(y_train).shape[0]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_train) 

Y_train = np_utils.to_categorical(y_train, num_classes) 
Y_test = np_utils.to_categorical(y_test, num_classes) 


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)  #Для предотвращения переобучения
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding - сохранять
#размер исходного изображения, рисунок дополняется нулями по краям. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=0, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

313/313 [==============================] - 1s 3ms/step - loss: 649.9954 - accuracy: 0.2454


[649.9954223632812, 0.24539999663829803]